# 素数

## 素数列挙

[エラトステネスの篩](https://ja.wikipedia.org/wiki/%E3%82%A8%E3%83%A9%E3%83%88%E3%82%B9%E3%83%86%E3%83%8D%E3%82%B9%E3%81%AE%E7%AF%A9) により素数を列挙する。  
エラトステネスの篩は、数列から「$2$の倍数」「$3$の倍数」$\ldots$ 「$n$の倍数」をこの順に取り除き、残った数を列挙する手法である。

計算量は$\mathcal{O}(n\log{\log{n}})$。

In [1]:
def sieve(n):
    """
    n以下の素数を昇順に列挙する
    """
    is_prime = [True for i in range(n+1)]
    primes = []
    for i in range(2, n+1):
        if is_prime[i]:
            primes.append(i)
            for j in range(i+i, n+1, i):
                is_prime[j] = False
    return primes

In [2]:
print(sieve(37))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]


## 素数判定

ある値が素数かどうかは、[試し割り法](https://ja.wikipedia.org/wiki/%E8%A9%A6%E3%81%97%E5%89%B2%E3%82%8A%E6%B3%95) で判定できる。  

### アルゴリズム

試し割り法では、1から順に整数 $p$ を取り出し、$n$ をその数で割り切れるかどうか実際に試す。  
「$n$ より小さい整数 $p$ が $n$ の倍数かどうかを、小さい順に確かめる」と読み替えてもよい。

ここで、素数判定に用いる除数の範囲を考える。  
例えば $31$ を整数 $p$ で割って商 $q$ を得るとき、$p$ が小さい順に

$$
\begin{eqnarray}
31 \div 2 &=& 16 \ldots 1 && (p=2, q=16) \\
31 \div 3 &=& 10 \ldots 1 && (p=3, q=10) \\
31 \div 4 &=& 7 \ldots 3 && (p=4, q=7) \\
31 \div 5 &=& 6 \ldots 1 && (p=5, q=6) \\
31 \div 6 &=& 5 \ldots 1 && (p=6, q=5) \\
31 \div 7 &=& 4 \ldots 3 && (p=7, q=4) \\
31 \div 8 &=& 3 \ldots 7 && (p=8, q=3) \\
31 \div 9 &=& 3 \ldots 4 && (p=9, q=3) \\
31 \div 10 &=& 3 \ldots 1 && (p=10, q=3) \\
&\vdots& \\
\end{eqnarray}
$$

と順に割っていくことになる。  
ここで、$p > q$ の範囲に着目する。「$6$ で割り切れるかどうか」は、その商が $5$ であることから、「$5$ で割り切れるかどうか」がわかればそれと同じ判定結果が得られる。  
同様に $7$ 以上の除数についても、商は除数より小さくなるから、その商で割り切れたかどうかで判定できたことになる。  

また、試し割りは素数のみ試せばよい。 これは、ある合成数で割り切れるとき、その素因数でも割り切れるからである。

一般に素数 $p \in [1, \sqrt{n}]$ を調べれば、全ての範囲で素数判定を行ったとみなせる。これにより、エラトステネスの篩の大きさの2乗まで素数判定が可能となる。

### 実装

In [3]:
def trial_division(n, primes):
    for p in primes:
        if p * p > n:
            break
        if n % p == 0:
            return False
    return True

In [4]:
primes = sieve(10**5)
print(trial_division(2010, primes))

False


In [5]:
print(trial_division(10**9+7, primes))

True


## 素因数分解

素因数分解についても、試し割り法で求めることができる。  
素数判定では単に「ある素数で割れるか」を調べていたが、「ある素数で何回割れるか」まで調べれば素因数とその個数がわかる。

計算量はクエリ毎に $\mathcal{O}(\pi(2^{d/2}))$ 。ここで $d$ は $n$ の2進表現における桁数、$\pi(x)$ は[素数計数関数](https://ja.wikipedia.org/wiki/%E7%B4%A0%E6%95%B0%E8%A8%88%E6%95%B0%E9%96%A2%E6%95%B0)である。  
nが32ビット符号なし整数のとき、試し割りの回数は最大で $\pi(2^{32/2}) = 6542$ 程度である。

In [6]:
from collections import defaultdict


def prime_factorization(n, primes):
    """
    nの素因数分解を求める
    """
    prime_factors = defaultdict(int)
    if n < 2:
        return prime_factors
    for p in primes:
        if p * p > n:
            break
        while n % p == 0:
            prime_factors[p] += 1
            n //= p
    if n > 1:
        prime_factors[n] = 1
    return prime_factors

In [7]:
from operator import mul
from functools import reduce

primes = sieve(10**5)
factors = prime_factorization(12345678, primes)
print(factors)
print(reduce(mul, [p**n for p, n in factors.items()]))

defaultdict(<class 'int'>, {2: 1, 3: 2, 47: 1, 14593: 1})
12345678


In [8]:
factors = prime_factorization(10**9+7, primes)
print(factors)
print(reduce(mul, [p**n for p, n in factors.items()]))

defaultdict(<class 'int'>, {1000000007: 1})
1000000007
